In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")
from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

from torch.optim import Adam

In [2]:
### Parameter for training

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   

args.to_save_path     = "/home/lukasprobst/Documents/Bachelorarbeit/Run_logs"           
args.freq_save_path   = "/home/lukasprobst/Documents/Bachelorarbeit/Freq_data"
args.window_save_path = "/home/lukasprobst/Documents/Bachelorarbeit/Sliding_window"
args.root_path        = "/home/lukasprobst/Documents/Bachelorarbeit/datasets"

args.drop_transition  = False
# Nomalization of the data, depends on dataset!
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"

args.batch_size       = 64 # Reduce if not enough graphic memory
args.shuffle          = True # Give NN more variety to learn
args.drop_last        = True # Drop last batch if smaller than batch size
args.train_vali_quote = 0.90 # 90% training, 10% validation                     

# training setting 
args.train_epochs            = 60

args.learning_rate           = 0.001 # 10^(-3)
args.learning_rate_patience  = 20 # Reduce learning rate every 20 epochs
args.learning_rate_factor    = 0.1 # Reduce learning rate by 10%

args.early_stop_patience     = 15 # Stop after 15 epochs without improvement

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0 # Use first GPU
args.use_multi_gpu           = False # Use multiple GPUs

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

# From ShaSpec paper:
# - 60 epochs
# - batch norm and dropout
# - adam optimizer with 10^(-2) weight decay
# - initial learning rate 10^(-3) and reduced by 10% every 20 epochs.

In [3]:
# Get content of folder args.window_save_path
files = os.listdir(args.window_save_path)

In [4]:
### Parameter for data

args.seed                             = 2 # Seed for reproducibility

args.data_name                        =  "dsads"

# Wavelet calculates frequence parameters from time series data, don't need that
args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable      = False
args.wavelet_filtering_layernorm      = False

args.regulatization_tradeoff          = 0
args.number_wavelet_filtering         = 6

# Augmentation difference
args.difference          =  False

# Augmentation filtering
args.filtering           =  False

# Augmentation  magnitude
args.magnitude           =  False
args.weighted_sampler    = False

# MIXUP
args.mixup_probability  = 0.5
args.mixup_alpha  = 0.5

# Random Augmentation
args.random_augmentation_prob = 0.5
args.random_augmentation_config = {"jitter":True,
                                   "moving_average":True,
                                   "magnitude_scaling":True,
                                   "magnitude_warp":True,
                                   "magnitude_shift":True,
                                   "time_warp":True,
                                   "window_warp":True,
                                   "window_slice":True,
                                   "random_sampling":True,
                                   "slope_adding":True
                                   }
random_augmentation_nr = 0
for key in args.random_augmentation_config.keys():
    if args.random_augmentation_config[key]:
        random_augmentation_nr = random_augmentation_nr+1
args.random_augmentation_nr = random_augmentation_nr
args.max_aug = 3

args.pos_select       = None
args.sensor_select    = None

args.representation_type = "time"
args.exp_mode            = "LOCV"
if args.data_name      ==  "skodar":
    args.exp_mode            = "SOCV"
    
config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.classes_num     =  config["num_classes"]
args.modalities_num  =  config["num_modalities"]
window_seconds       = config["window_seconds"]
print
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# Input information
# For ShaSpec we want the number of channels per modality
args.c_in            =  config["num_channels"] // args.modalities_num

print(args.c_in )
if args.difference:
    args.c_in  = args.c_in * 2
    print(args.c_in )
if args.filtering:
    for col in config["sensors"]:
        if "acc" in col:
            args.c_in = args.c_in+1
            print(args.c_in )

if args.wavelet_filtering :
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1
            

9


In [5]:
args.c_in

9

In [6]:
args.classes_num

19

In [7]:
### Parameter for the model

args.model_type = "shaspec"

args.activation = "ReLU"
args.decoder_type = "FC"
args.shared_encoder_type = "concatenated"

exp = Exp(args)

Use GPU: cuda:0
Build the ShaSpec model!
Done!
Parameter : 366517
Set the seed as :  2


In [8]:
exp.model

model_builder(
  (model): ShaSpec(
    (specific_encoders): ModuleList(
      (0-4): 5 x SpecificEncoder(
        (conv_layers): Sequential(
          (0): Conv2d(1, 42, kernel_size=(5, 1), stride=(2, 1))
          (1): ReLU()
          (2): Conv2d(42, 42, kernel_size=(5, 1), stride=(2, 1))
          (3): ReLU()
          (4): Conv2d(42, 42, kernel_size=(5, 1), stride=(2, 1))
          (5): ReLU()
          (6): Conv2d(42, 42, kernel_size=(5, 1), stride=(2, 1))
          (7): ReLU()
        )
        (sa): SelfAttention(
          (query): Conv1d(42, 5, kernel_size=(1,), stride=(1,), bias=False)
          (key): Conv1d(42, 5, kernel_size=(1,), stride=(1,), bias=False)
          (value): Conv1d(42, 42, kernel_size=(1,), stride=(1,), bias=False)
        )
        (fc_fusion): Linear(in_features=210, out_features=84, bias=True)
      )
    )
    (shared_encoder): SharedEncoder(
      (conv_layers): Sequential(
        (0): Conv2d(1, 42, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU

In [9]:
exp.train()

 ----------------------- load all the data -------------------
-----------------------Sliding file are generated -----------------------
-----------------------Sliding file are generated -----------------------
================ LOCV Mode ====================
================ 8 CV ======================
================ the 0 th CV Experiment ================ 
Leave one Out Experiment : The 1 Part as the test
[-] Target sampling weights:  [0.00263158 0.00265957 0.00266667 0.00269542 0.00257732 0.00265252
 0.00271739 0.00263852 0.0026178  0.00263852 0.00265252 0.00265957
 0.00271003 0.0027027  0.0025974  0.00261097 0.00260417 0.00261097
 0.00263158]
Train data number :  7182
The number of classes is :  19
The input_length  is :  125
The channel_in is :  45
Validation data number :  798
Test data number :  1140
SKIP FINE TUNING?  False
================ Build the model ================ 
Build the ShaSpec model!
-----------------


Training Process [epochs]:   0%|          | 0/60 [01:15<?, ?it/s]


KeyboardInterrupt: 